## Imports

In [1]:
import numpy as np
import random
import operator
import time
import pandas as pd
from bandle_functions import *

## Creating bandle_official_* (word length) lists

In [2]:
### Official list as placeholder for now
official_words = []

with open("data/official_words_processed.txt", "r", encoding = "utf-8") as f:
    for word in f.read().split("\n"):
        official_words.append(word)

f.close() # closes connection to file

print(len(official_words))
official_words[:5]

2310


['wince', 'thyme', 'mower', 'horde', 'heard']

In [3]:
### list creation here

## `bandle_user()` function

In [4]:
# def bandle_user(word_list: list, max_guesses: int = None, 
#                   guess: str = None, target: str = None,
#                   random_guess: bool = False, random_target: bool = False, 
#                   hints: bool = False, drama: float = None, 
#                   return_stats: bool = False, record: bool = False, hf_mod: bool = True):
#     """
#     Mimicking the popular web game, this function matches a current word to a target word automatically, in the most statistically optimal way possible.

#     Parameters:
#     ------
#     `word_list`: list
#         list of valid words to be considered
#     `guess`: str
#         a string -- must be the same length as `target_word`
#     `target`: str
#         a string -- must be the same length as `opening_word`
#     `max_guesses`: int
#         the maximum number of attempts allowed to solve the Bandle
#     `random_guess`: bool
#         if True, randomly chooses a starting word from all words within `word_list`. If False, passed starting word must be used instead
#     `random_target`: bool
#         if True, randomly chooses a target word from all words within `word_list`. If False, passed target word must be used instead
#     `hints`: bool
#         if True, provides insight into user's current progress solving the puzzle, displaying remaining possible words and ranking them in order of guess quality
#     `drama`: float or int
#         if int provided, each guess' output is delayed by that number of seconds, else each output is shown as quickly as possible. For ~dRaMaTiC eFfEcT~
#     `return_stats`: bool
#         if True, prints nothing and returns a dictionary of various statistics about the function's performance trying to solve the puzzle
#     `record`: bool
#         if True, creates a .txt file with the same information printed according to the indicated verbosity

#     Returns:
#     ------
#     `stats_dict`: dict
#         dictionary containing various statistics about the function's performance trying to solve the puzzle
#     `record_list`: list
#         List of things to print -- used for particular output in HuggingFace frontend web app
#     """

#     # user input for first guess

#     guess = input("Please enter a starting guess")

#     guess = guess.lower()
#     target = target.lower()

#     sugg_words = []

#     for i in range(0, 20):
#         ran_int = random.randint(0, len(word_list) - 1)
#         word = word_list[ran_int]
#         sugg_words.append(word)

#     if guess not in word_list:
#         print ("Guess word not in passed word list.\nOnly words within the given word list are valid.")
#         print (f"Here are some examples of valid words from the passed word list.\n\t{sugg_words[:10]}")
#         return None
    
#     if target not in word_list:
#         print ("Target word not in passed word list.\nOnly words within the given word list are valid.")
#         print (f"Here are some examples of valid words from the passed word list.\n\t{sugg_words[-10:]}")
#         return None

#     if random_guess == True:
#         randomint_guess = random.randint(0, len(word_list) - 1)
#         guess = word_list[randomint_guess]

#     if random_target == True:
#         randomint_target = random.randint(0, len(word_list) - 1)
#         target = word_list[randomint_target]

#     stats_dict = {}
#     stats_dict['first_guess'] = guess
#     stats_dict['target_word'] = target
#     stats_dict['first_guess_vowels'] = float(count_vows_cons(guess, y_vow = True)['vows'])
#     stats_dict['first_guess_consonants'] = float(count_vows_cons(guess, y_vow = True)['cons'])
#     stats_dict['target_vowels'] = float(count_vows_cons(target, y_vow = True)['vows'])
#     stats_dict['target_consonants'] = float(count_vows_cons(target, y_vow = True)['cons'])
    
#     # get entropy of the first guess word and target word in the entire word_list
#     for tup in get_word_entropy(word_list, word_list, normalized = True):
#         if tup[0] == guess:
#             stats_dict['first_guess_entropy'] = tup[1]
#         if tup[0] == target:
#             stats_dict['target_entropy'] = tup[1]

#     guess_entropies = []
#     guess_entropies.append(stats_dict['first_guess_entropy'])

#     # luck_guess_1 = round(1 - ((1 / len(word_list)) * guess_entropies[0] / 100), 2) * 100

#     english_alphabet = "abcdefghijklmnopqrstuvwxyz"

#     word_list_sorted_counts = get_letter_counts(english_alphabet, word_list, sort = "descending")
    
#     Bandlen = len(guess)
#     letter_positions = set(i for i in range(0, Bandlen))

#     guess_set = set()
#     perfect_dict = {}
#     wrong_pos_dict = {}
#     wrong_pos_set = set()
#     dont_guess_again = set()

#     guessed_words = [] # running set of guessed words
#     guess_num = 0 # baseline for variable
#     dont_guess_words = set()
#     incorrect_positions = []
#     reduction_per_guess = []

#     if max_guesses == None: # if no value is passed, default is len(guess)
#         max_guesses = Bandlen
#     else: # else it is the value passed
#         max_guesses = max_guesses

#     perfect_letts_per_guess = []
#     wrong_pos_per_guess = []
#     wrong_letts_per_guess = []

#     record_list = []

#     while guess: # while there is any guess -- there are conditions to break it at the bottom

#         guess_num += 1

#         guessed_words.append(guess)

#         if drama:
#             time.sleep(drama)

#         # guess_num += 1 # each time the guess is processed
#         if return_stats == False:
#             if guess_num == 1:
#                 print("-----------------------------\n")
#                 record_list.append("-----------------------------\n")
    
#         if return_stats == False:
#             print(f"Guess {guess_num}: '{guess}'")
#             record_list.append(f"Guess {guess_num}: '{guess}'")

#         if guess == target:
#             stats_dict['target_guessed'] = True
#             if return_stats == False:
#                 if guess_num == 1:
#                     print(f"Congratulations! The Bandle has been solved in {guess_num} guess, that's amazingly lucky!")
#                     print(f"The target word was '{target}'")
#                     record_list.append(f"Congratulations! The Bandle has been solved in {guess_num} guess, that's amazingly lucky!")
#                     record_list.append(f"The target word was '{target}'.")
#                     print("\n-----------------------------\n")
#                     record_list.append("-----------------------------\n")
#                     perfect_letts_per_guess.append(5)
#                     wrong_pos_per_guess.append(0)
#                     wrong_letts_per_guess.append(0)
#             break

#         guess_set = set()
#         wrong_pos_set = set()

#         #### Step 2 -- ALL PERFECT
#         for i in letter_positions: # number of letters in each word (current word and target word)
#             guess_set.add(guess[i])

#             if guess[i] not in perfect_dict:
#                 perfect_dict[guess[i]] = set()
#             if guess[i] not in wrong_pos_dict:
#                 wrong_pos_dict[guess[i]] = set()

#             ### EVALUATE CURRENT GUESS
#             if guess[i] == target[i]: # letter == correct and position == correct
#                 perfect_dict[guess[i]].add(i)

#             if (guess[i] != target[i] and  guess[i] in target): # letter == correct and position != correct
#                 wrong_pos_dict[guess[i]].add(i)
#                 wrong_pos_set.add(guess[i])

#             if guess[i] not in target: # if letter is not relevant at all
#                 dont_guess_again.add(guess[i])

#         #### Step 3 -- ALL PERFECT
#         next_letters = set()
#         for letter, positions in perfect_dict.items():
#             if len(positions) > 0:
#                 next_letters.add(letter)

#         for letter, positions in wrong_pos_dict.items():
#             if len(positions) > 0:
#                 next_letters.add(letter)

#         #### List of tuples of correct letter positions in new valid words. Eg: [('e', 2), ('a', 3)]
#         perfect_letters = []
#         for letter, positions in perfect_dict.items():
#             for pos in positions:
#                 if len(positions) > 0:
#                     perfect_letters.append((letter, pos))

#         #### all words that have correct letters in same spots
#         words_matching_correct_all = []
#         for word in word_list:
#             word_set = set()
#             for letter, pos in perfect_letters:
#                 if pos < len(word):
#                     if word[pos] == letter:
#                         words_matching_correct_all.append(word)

#         #### excluding words with letters in known incorrect positions
#         for letter, positions in wrong_pos_dict.items():
#             for pos in positions:
#                 if len(positions) > 0:
#                     if (letter, pos) not in incorrect_positions:
#                         incorrect_positions.append((letter, pos))

#         # sorting lists of tuples just to make them look nice in the printout
#         incorrect_positions = sorted(incorrect_positions, key = operator.itemgetter(1), reverse = False)
#         perfect_letters = sorted(perfect_letters, key = operator.itemgetter(1), reverse = False)

#         #### all words that have correct letters in incorrect spots -- so they can be excluded efficiently
        
#         # print(incorrect_positions)
        
#         for word in word_list:
#             word_set = set()
#             for letter, pos in incorrect_positions:
#                 if pos < len(word):
#                     if word[pos] == letter:
#                         dont_guess_words.add(word)
#         for word in word_list:
#             word_set = set()
#             for letter, pos in incorrect_positions:
#                 if pos < len(word):
#                     if word[pos] == letter:
#                         dont_guess_words.add(word)

#         for bad_letter in dont_guess_again:
#             for word in word_list:
#                 if (bad_letter in word and word not in dont_guess_words):
#                     dont_guess_words.add(word)

#         if return_stats == False:
#             # if hints == True:
#             print(f"Letters in correct positions:\n\t{perfect_letters}\n")
#             print(f"Letters in incorrect positions:\n\t{incorrect_positions}\n")
#             print (f"Letters to guess again:\n\t{sorted(list(next_letters), reverse = False)}\n")
#             print(f"Letters to not guess again:\n\t{sorted(list(dont_guess_again), reverse = False)}\n") # works
            
#             if len(perfect_letters) == 0:
#                 record_list.append(f"Letters in correct positions: None\n")
#             else:
#                 record_list.append(f"Letters in correct positions: {perfect_letters}\n")
            
#             if len(incorrect_positions) == 0:
#                 record_list.append(f"Letters in incorrect positions: None\n")
#             else:
#                 record_list.append(f"Letters in incorrect positions: {incorrect_positions}\n")
            
#             if len(next_letters) == 0:
#                 record_list.append(f"Letters to guess again: None\n")
#             else:
#                 record_list.append(f"Letters to guess again: {sorted(list(next_letters), reverse = False)}\n")
#                 # if
#                 #     record_list.append(f"Letters to not guess again: {sorted(list(dont_guess_again), reverse = False)}\n") # works

#         # Returns True
#         # print(A.issubset(B)) # "if everything in A is in B", returns Bool

#         perfect_letts_per_guess.append(len(perfect_letters))
#         wrong_pos_per_guess.append(len(incorrect_positions))
#         wrong_letts_per_guess.append(len(dont_guess_again))

#         potential_next_guesses = set()
#         middle_set = set()

#         if len(perfect_letters) == 0 and len(incorrect_positions) == 0: # if there are NEITHER perfect letters, NOR incorrect positions, ....
#             for word in word_list:
#                 if word not in dont_guess_words:
#                     if word not in guessed_words:
#                         potential_next_guesses.add(word)
                                        
#             # print(f"GUESS {guess_num} : TEST 1-1")

#         if len(perfect_letters) == 0 and len(incorrect_positions) != 0: # if there are no perfect letters whatsoever, but there ARE incorrect positions ....
#             for word in word_list:
#                 for incor_letter, incor_pos in incorrect_positions:
#                     if incor_pos < len(word):
#                         if word[incor_pos] != incor_letter:
#                             if word not in dont_guess_words: # just in case
#                                 word_set = set()
#                                 for letter in word:
#                                     word_set.add(letter)
    
#                                     if next_letters.issubset(word_set):
#                                         if word not in guessed_words:
#                                             if len(dont_guess_again) > 0:
#                                                 for bad_letter in dont_guess_again:
#                                                     if bad_letter not in word:
#                                                         # potential_next_guesses.append(word)
#                                                         potential_next_guesses.add(word)
#                                             else:
#                                                 potential_next_guesses.add(word)
            
#             # print(f"GUESS {guess_num} : TEST 2-1")

#         else:
#             for word in word_list:
#                 if word not in dont_guess_words: # just in case
#                     word_set = set()
#                     for letter in word:
#                         word_set.add(letter)
#                         if next_letters.issubset(word_set):
#                             if word not in guessed_words:
#                                 # print ("TEST 3-2")

#                                 if len(dont_guess_again) > 0:
#                                     for bad_letter in dont_guess_again:
#                                         if bad_letter not in word:
#                                             middle_set.add(word)
#                                 else:
#                                     middle_set.add(word)
#             for word in middle_set:
#                 dummy_list = []
#                 for good_lett, good_pos in perfect_letters:
#                     if word[good_pos] == good_lett:
#                         dummy_list.append(1)
#                         if len(dummy_list) == len(perfect_letters):
#                             potential_next_guesses.add(word)
#             for word in middle_set:
#                 dummy_list = []
#                 for bad_lett, bad_pos in incorrect_positions:
#                     if bad_pos < len(word):
#                         if word[bad_pos] == bad_lett:
#                             dummy_list.append(1)
#                             if len(dummy_list) > 0:
#                                 potential_next_guesses.remove(word)
                                        
#             # print(f"GUESS {guess_num} : TEST 3-1")

#         if return_stats == False:
#             if hints == True:
#                 print(f"At this point:")
#                 print(f"\t{len(word_list) - len(potential_next_guesses)}, {round((len(word_list) - len(potential_next_guesses)) / len(word_list) * 100, 2)}% of total words have been eliminated, and")
#                 print(f"\t{len(potential_next_guesses)}, {round(len(potential_next_guesses) / len(word_list) * 100, 2)}% of total words remain possible.\n")
#                 # record_list.append(f"At this point:")
#                 record_list.append(f"{len(word_list) - len(potential_next_guesses)} ({round((len(word_list) - len(potential_next_guesses)) / len(word_list) * 100, 2)}% of all) words have been eliminated, and {len(potential_next_guesses)} ({round(len(potential_next_guesses) / len(word_list) * 100, 2)}% of all) words are still possible.\n")
        
#         reduction_per_guess.append(len(potential_next_guesses))
                
#         #### Guessing next word
#         if len(potential_next_guesses) == 1:

#             if return_stats == False:
#                 if hints == True:
#                     # print(f"All potential next guesses:\n\t{word_ratings}\n")
#                     # print(f"Words guessed so far:\n\t{guessed_words}.\n")
#                     # record_list.append(f"Potential next guesses: {word_ratings}\n")
#                     record_list.append(f"Words guessed so far: {guessed_words}.\n")
#                     # print(f"The only remaining possible word is:\n\t'{list(potential_next_guesses)[0]}'\n")
#                     record_list.append(f"The only remaining possible word is: '{list(potential_next_guesses)[0]}'\n")
                
#             # guess = list(potential_next_guesses)[0]
#         # if len(guessed_words) == (max_guesses - 1):
#         #     guess = input("Please enter your final guess")
#         # else:
#         #     guess = input("Please enter your next guess")
            
#         # if guess not in word_list:
#         #     print ("Guess word not in passed word list.\nOnly words within the given word list are valid.")
#         #     print (f"Here are some examples of valid words from the passed word list.\n\t{sugg_words[:10]}")
#         # return None
#         guess_entropies.append(get_word_entropy([guess], word_list, normalized = True, ascending = False)[0][1])

#         # else:

#         best_next_guesses = list(potential_next_guesses)                
#         #     # print (best_next_guesses)
            
#         word_ratings = get_word_entropy(best_next_guesses, word_list, normalized = True, ascending = False) # "internal" ratings
# ##### All of this is moot because user will supply next guess #####

#             # # Get max rating of all words
#             # max_rating = -np.inf
#             # for word, rating in word_ratings:
#             #     if rating > max_rating:
#             #         max_rating = rating

#             # # add best rated words (all equally best entropy in next guess list) to set
#             # best_of_the_best_1 = []
#             # for word, rating in word_ratings:
#             #     if rating == max_rating:
#             #         best_of_the_best_1.append(word)

#             # # only using top ten most frequent prefixes suffixes to bias. After that it the impact is especially negligible
#             # test_starts = get_gram_freq(word_list = word_list, letters_length = 1, position = "start", search = None)[:10]
#             # test_ends = get_gram_freq(word_list = word_list, letters_length = 1, position = "end", search = None)[:10]

#             # # list of the best words that also have the most frequent starting and ending letters (suffixes and prefixes didn't have an impact)
#             # best_of_the_best_2 = []
#             # for start_gram, start_count in test_starts:
#             #     for end_gram, end_count in test_ends:
#             #         for word in best_of_the_best_1:
#             #             if word[:1] == start_gram and word[-1:] == end_gram:
#             #                 best_of_the_best_2.append(word)

#             # if len(best_of_the_best_2) > 0:
#             #     guess = best_of_the_best_2[0]
#             # else:
#             #     guess = best_of_the_best_1[0] # they're all equally the best of the best possible guesses so just pick the first

# ##### All of this is moot because user will supply next guess #####


#         if return_stats == False:
#             if hints == True:
#                 if len(word_ratings) <= 40:
#                     print(f"All potential next guesses:\n\t{word_ratings}\n")
#                     print(f"Words guessed so far:\n\t{guessed_words}.\n")
#                     record_list.append(f"Potential next guesses: {word_ratings}\n")
#                     record_list.append(f"Words guessed so far: {guessed_words}.\n")
#                 else:
#                     print(f"The top 40 potential next guesses are:\n\t{word_ratings[:40]}\n")
#                     print(f"Words guessed so far:\n\t{guessed_words}.\n")
#                     record_list.append(f"The top 40 potential next guesses are: {word_ratings[:40]}\n")
#                     record_list.append(f"Words guessed so far: {guessed_words}.\n")
        
#         if len(guessed_words) == (max_guesses - 1):
#             guess = input("Please enter your final guess")
#         else:
#             guess = input("Please enter your next guess")
            
#         if guess not in word_list:
#             print ("Guess word not in passed word list.\nOnly words within the given word list are valid.")
#             print (f"Here are some examples of valid words from the passed word list.\n\t{sugg_words[:10]}")

#         guess_entropies.append(get_word_entropy([guess], word_list, normalized = True, ascending = False)[0][1])

#         #### Guess has now been made -- what to do next
#         if guess_num == max_guesses: # if at max guesses allowed
#             guessed_words.append(guess)
#             stats_dict['target_guessed'] = False
#             if return_stats == False:
#                 if hints == True:
#                     # print("-----------------------------\n")
#                     print(f"Unfortunately, the Bandle could not be solved in {max_guesses} guesses.\n")
#                     print(f"The target word was '{target}'. Better luck next time!\n")
#                     print("-----------------------------\n")
#                     record_list.append(f"Unfortunately, the Bandle could not be solved in {max_guesses} guesses.\n")
#                     record_list.append(f"The target word was '{target}'. Better luck next time!\n")
#                     record_list.append("-----------------------------\n")
#                 else:
#                     print(f"\nUnfortunately, the Bandle could not be solved in {max_guesses} guesses.")
#                     print(f"The target word was '{target}'. Better luck next time!\n")
#                     record_list.append(f"Unfortunately, the Bandle could not be solved in {max_guesses} guesses.")
#                     record_list.append(f"The target word was '{target}'. Better luck next time!\n")
#             break
#         else: # if not at max guesses yet allowed
#             # stats_dict['target_guessed'] = False
#             if return_stats == False:
#                 # if hints == True:
#                 print(f"Next guess:\n\t'{guess}'")
#                 print("\n-----------------------------\n")
#                 record_list.append(f"Next guess: '{guess}'")
#                 record_list.append("-----------------------------\n")

#         if guess == target:
#             guess_num += 1
#             guessed_words.append(guess)
#             stats_dict['target_guessed'] = True

#             if return_stats == False:
#                 print(f"Guess {guess_num}: '{guess}'\n")
#                 print(f"Congratulations! The Bandle has been solved in {guess_num} guesses!")
#                 record_list.append(f"Guess {guess_num}: '{guess}'\n")
#                 record_list.append(f"Congratulations! The Bandle has been solved in {guess_num} guesses!")

#                 if max_guesses - guess_num == 0:
#                     print(f"Lucky! It was the last guess.")
#                     record_list.append(f"Lucky! It was the last guess.")
#                 else:
#                     print(f"There were still {max_guesses - guess_num} guesses remaining.")
#                     record_list.append(f"There were still {max_guesses - guess_num} guesses remaining.")

#             if return_stats == False:   
#                 # stats_dict['target_guessed'] = True                 
#                 print(f"\nThe target word was '{target}'.")
#                 print("\n-----------------------------")
#                 record_list.append(f"The target word was '{target}'.")
#                 record_list.append("-----------------------------")
#             break

#     #### STATS STUFF    
#     mid_guesses_vows = 0
#     mid_guesses_cons = 0
#     avg_perf_letters = 0
#     avg_wrong_pos_letters = 0
#     avg_wrong_letters = 0

#     for i, word in enumerate(guessed_words):
#         mid_guesses_vows += count_vows_cons(word, y_vow = True)['vows']
#         mid_guesses_cons += count_vows_cons(word, y_vow = True)['cons']
        
#     for i in range(0, len(guessed_words) - 1):
#         avg_perf_letters += perfect_letts_per_guess[i]
#         avg_wrong_pos_letters += wrong_pos_per_guess[i]
#         avg_wrong_letters += wrong_letts_per_guess[i]

#     stats_dict['mid_guesses_avg_vows'] = float(round(mid_guesses_vows / len(guessed_words), 2))
#     stats_dict['mid_guesses_avg_cons'] = float(round(mid_guesses_cons / len(guessed_words), 2))

#     stats_dict['avg_perf_letters'] = float(round(np.mean(avg_perf_letters), 2))
#     stats_dict['avg_wrong_pos_letters'] = float(round(np.mean(avg_wrong_pos_letters), 2))
#     stats_dict['avg_wrong_letters'] = float(round(np.mean(avg_wrong_letters), 2))
    
#     # average number of words remaining after each guess -- the higher this is, the luckier the person got (the lower, the more guesses it took)
#     stats_dict['avg_remaining'] = float(round(np.mean(reduction_per_guess), 2))

#     # avg entropy of each guessed word relative to all other words possible at that moment -- this should consistently be 100 for the algorithm, but will be different for user
#     if len(guess_entropies) > 1: # in case of guessing it correctly on the first try
#         sum_entropies = 0
#         for entropy in guess_entropies:
#             sum_entropies += entropy

#         average_entropy = float(round(sum_entropies / len(guess_entropies), 2))
#         stats_dict['avg_intermediate_guess_entropy'] = average_entropy
#     else:
#         stats_dict['avg_intermediate_guess_entropy'] = float(100)

#     expected_guesses = 3.85

#     # guess_num = 3
#     # average_entropy = 95
#     luck = round(1 - ((((guess_num / expected_guesses) * (stats_dict['avg_intermediate_guess_entropy'] / 100)) / max_guesses) * 5), 2)
#     stats_dict['luck'] = luck

#     if record == True:
#         if hints == True:
#             with open(f"solutions/{guessed_words[0]}_{target}_wizard_detailed.txt", "w") as fout:
#                 for line in record_list:
#                     fout.write(line + "\n") # write each line of list of printed text to .txt file
#         else:
#             with open(f"solutions/{guessed_words[0]}_{target}_wizard_summary.txt", "w") as fout:
#                 for line in record_list:
#                     fout.write(line + "\n") # write

#     # if guess_num <= len(guess):
#     if guess_num <= 6:
#         stats_dict['valid_success'] = True
#     else:
#         stats_dict['valid_success'] = False

#     stats_dict['num_guesses'] = float(guess_num)

#     # if return_stats == True:
#     #     return stats_dict
#     if hf_mod == True:
#         return record_list

In [5]:
bandle_user(word_list = official_words, max_guesses = 6, 
                guess = "paint", target = "force",
                random_guess = False, random_target = False, 
                hints = False, drama = 0, return_stats = False,
                record = False, hf_mod = False)

-----------------------------

Guess 1: 'shirt'
Letters in correct positions:
	[]

Letters in incorrect positions:
	[('r', 3)]

Letters to guess again:
	['r']

Letters to not guess again:
	['h', 'i', 's', 't']

Next guess:
	'heart'

-----------------------------

Guess 2: 'heart'
Letters in correct positions:
	[]

Letters in incorrect positions:
	[('e', 1), ('r', 3)]

Letters to guess again:
	['e', 'r']

Letters to not guess again:
	['a', 'h', 'i', 's', 't']

Next guess:
	'later'

-----------------------------

Guess 3: 'later'
Letters in correct positions:
	[]

Letters in incorrect positions:
	[('e', 1), ('r', 3), ('e', 3), ('r', 4)]

Letters to guess again:
	['e', 'r']

Letters to not guess again:
	['a', 'h', 'i', 'l', 's', 't']

Next guess:
	'force'

-----------------------------

Guess 4: 'force'

Congratulations! The Bandle has been solved in 4 guesses!
There were still 2 guesses remaining.

The target word was 'force'.

-----------------------------
